<a href="https://colab.research.google.com/github/janghyukjin/MachineLearning/blob/master/Assignment11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
ls

In [0]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
nltk.download('wordnet')
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

review_data = load_files(r"movie_review")

X, y = review_data.data, review_data.target
print(len(X))
print(len(y))
documents = []

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()
print(len(X))
print(len(y))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)


In [0]:
u=np.random.randn(120,1500) 
v=np.random.randn(10,120) 
w=np.random.randn(1,10) 
m = 1400
test_m = 600
cost = 0
test_cost = 0
accuracy = 0
test_accuracy = 0
cost_list = np.empty(0)
accuracy_list = np.empty(0)
test_cost_list = np.empty(0)
test_accuracy_list = np.empty(0)
bias1 = np.zeros((120, 1))
bias2 = np.zeros((10, 1)) 
bias3 = np.zeros((1, 1))
lamda = 0.5
cnt = 0
print(y_train)
print(len(X_train))
print(len(X_test))

print(len(y_train))
print(len(y_test))
def sigmoid(z):
  return 1/(1+np.exp(-z))

In [0]:
alpha = 0.5
count = 0 
final_cnt = 0
jx = np.empty(0)
while(count<10000):
  if count%10==0:
    print(count)
  jx = np.append(jx, count)
  count += 1
  
  #propagation

  bias_y = np.dot(u, X_train.T)

  bias_y += bias1 
  sig_y = sigmoid(bias_y)
  bias_z = np.dot(v, sig_y)
  bias_z += bias2 
  sig_z = sigmoid(bias_z)
  bias_h = np.dot(w, sig_z)
  bias_h += bias3
  sig_h = sigmoid(bias_h)
  
  #objective Function
  cost = (-1/1400)*np.sum(y_train*np.log(sig_h) + (1-y_train)*np.log(1-sig_h)) + (lamda/(2*1000))*(np.sum(np.square(u)) + np.sum(np.square(v)) + np.sum(np.square(w)))
  cost_list = np.append(cost_list,cost)
  
  #Calculate Accuracy
  accuracy = 0

  for i in range(1401):
    if sig_h[0][i]>=0.5:
      sig_h[0][i] = 1
    else:
      sig_h[0][i] = 0
    if y_train[i] == sig_h[0][i]:
      accuracy += 1
  accuracy/=1400
  accuracy*=100 
  accuracy_list = np.append(accuracy_list, accuracy)

  #back propagation
  delta_h = (sig_h-y_train)
  delta_z = np.dot(np.transpose(w),delta_h)*sig_z*(1-sig_z)
  delta_y = np.dot(np.transpose(v),delta_z)*sig_y*(1-sig_y)

  delta_bias3 = np.sum(delta_h, axis=1, keepdims=True)/1000
  delta_bias2 = np.sum(delta_z, axis=1, keepdims=True)/1000
  delta_bias1 = np.sum(delta_y, axis=1, keepdims=True)/1000

  delta_w = np.dot(delta_h,np.transpose(sig_z))
  delta_v = np.dot(delta_z,np.transpose(sig_y))
  delta_u = np.dot(delta_y,X_train)
  delta_w/=1000
  delta_v/=1000
  delta_u/=1000
  delta_w += (lamda/1000)*w
  delta_v += (lamda/1000)*v

  delta_u += (lamda/1000)*u

  #update theta, bias
  u -= alpha * delta_u 
  v -= alpha * delta_v 
  w -= alpha * delta_w 
  bias1 -= alpha * delta_bias1 
  bias2 -= alpha * delta_bias2 
  bias3 -= alpha * delta_bias3

  #test images propagation
  test_bias_y = np.dot(u, X_test.T)
  test_bias_y += bias1 
  test_sig_y = sigmoid(test_bias_y)
  test_bias_z = np.dot(v, test_sig_y)
  test_bias_z += bias2 
  test_sig_z = sigmoid(test_bias_z)
  test_bias_h = np.dot(w, test_sig_z)
  test_bias_h += bias3
  test_sig_h = sigmoid(test_bias_h)
  #test images objective function
  test_cost = (-1/600)*np.sum(y_test*np.log(test_sig_h) + (1-y_test)*np.log(1-test_sig_h)) + (lamda/(2*9000))*(np.sum(np.square(u)) + np.sum(np.square(v)) + np.sum(np.square(w)))
  test_cost_list = np.append(test_cost_list,test_cost)
  #test images accuracy
  test_accuracy = 0

  for i in range(601):
    if test_sig_h[0][i]>=0.5:
      test_sig_h[0][i] = 1
    else:
      test_sig_h[0][i] = 0
    if y_test[i] == test_sig_h[0][i]:
      test_accuracy += 1
  test_accuracy/=600
  test_accuracy*=100 
  print(test_accuracy)
  test_accuracy_list = np.append(test_accuracy_list, test_accuracy)
  #change running rate for improving accuracy
  if test_accuracy > 80: 
    alpha = 0.8
  if test_accuracy > 88.5: 
    alpha = 0.5
  if test_accuracy > 89: 
    final_cnt += 1
  if final_cnt > 50:
    break
  if final_cnt>5:
    if test_accuracy<89:
      break